In [37]:
#basic python libraries
import numpy as np
import pandas as pd

#sklearn libraries
import sklearn
#train and test modules
from sklearn.model_selection import (train_test_split, cross_val_score, KFold)
#dummy classifier
from sklearn.dummy import DummyClassifier
# fill missing values with mean/median/mode
from sklearn.impute import SimpleImputer  
# turn categorical values into binary columns
from sklearn.preprocessing import OneHotEncoder
# turn str into categorical  
from sklearn.preprocessing import OrdinalEncoder
# binning
from sklearn.preprocessing import KBinsDiscretizer
# scaling 
from sklearn.preprocessing import MinMaxScaler, StandardScaler
#pipeline
from sklearn.pipeline import Pipeline

In [38]:
df = pd.read_csv('./data/train.csv')

In [39]:
y = df['Survived']  # DV must always be a single column
X = df.drop('Survived', axis=1, inplace=True)  # save df without 'Survived' in X for training
#the inplace is important on this one

In [40]:
#these are columns I don't think are useful particularly, I would probably drop more of them but w/e
X = df.drop(['Name', 'Ticket', 'Cabin', 'PassengerId'], axis = 1)

In [41]:
X.head() #idk why this one sometimes is fussy

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S


In [42]:
#standard xtrain y train for sklearn
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=14, stratify=y)

OK, I am going to create variables for each of the sk learn bits and then put it all straight into the column transform pipeline, this may not work

#this is the varible creating section, ideally all the variables will be created here for proper reference in the pipeline
#actually wait do I need this at all? it looks like you can call these straight from the pipeline
imputer = SimpleImputer(strategy='mean')
encoder = OneHotEncoder()
binning = KBinsDiscretizer()


In [43]:
from sklearn.pipeline import Pipeline

In [44]:
#this is what I will use in the column transform, but I wonder if this can be expanded, each pipelines is essentially limited to the type, though I wonder if you could make a function that sorts them a bit?

pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy = 'mean')),
    ('bins', KBinsDiscretizer())
    
])
one_encode = OneHotEncoder()


Column Transfer here

In [45]:
from sklearn.compose import ColumnTransformer

In [46]:
column_transformer = ColumnTransformer([
    ('one_encode', OneHotEncoder(),['Pclass', "Sex"]), #One Hot Encoder is for categorical/discrete variables to turn them into binary matrices 
    ('pipeline', pipeline, ['Age']), #Imputer fills in missing values with the mean here, and the bins for the even beinning
    ('pass', 'passthrough', ['SibSp', "Parch"]), #tbh, IDGAF about these 2 anyhow, if they find something it will be a surprise
       
]
,remainder = "drop")

In [47]:
#this is for cross validation stuff, but ngl, I am not sure how cleanly this xfers over from the penguin set
X_train_transformed = column_transformer.fit_transform(X_train)
X_train_df = pd.DataFrame(X_train_transformed,columns=column_transformer.get_feature_names_out())
X_train_df

,one_encode__Pclass_1,one_encode__Pclass_2,one_encode__Pclass_3,one_encode__Sex_female,one_encode__Sex_male,pipeline__Age_0.0,pipeline__Age_1.0,pipeline__Age_2.0,pipeline__Age_3.0,pipeline__Age_4.0,pass__SibSp,pass__Parch
0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
663,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
664,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
665,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
666,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,2.0


In [48]:
#now to actually input this all and see something? sticking with a log reg for week 2 theme, but in theory here input any supervised learning
from sklearn.linear_model import LogisticRegression 

general_pipeline = Pipeline([
    ('column_transformer', column_transformer),
    ("model", LogisticRegression())
    
])
#parentheses are important

In [49]:
#the log model and fit 
model = LogisticRegression()

model.fit(X_train_transformed, y_train)

LogisticRegression()

In [50]:
from sklearn.model_selection import train_test_split

general_pipeline.fit(X_train, y_train)

#did I make a dumb mistake? parentehses

Pipeline(steps=[('column_transformer',
                 ColumnTransformer(transformers=[('one_encode', OneHotEncoder(),
                                                  ['Pclass', 'Sex']),
                                                 ('pipeline',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('bins',
                                                                   KBinsDiscretizer())]),
                                                  ['Age']),
                                                 ('pass', 'passthrough',
                                                  ['SibSp', 'Parch'])])),
                ('model', LogisticRegression())])

In [51]:
general_pipeline.score(X_test, y_test)
#in theory better than the dummy but I will come back to this

0.7668161434977578

In [52]:
general_pipeline.predict(X_test);

In [53]:
general_pipeline.predict_proba(X_test);

Moving to cross validation with kfolds here, lets see

In [54]:
kf = KFold(n_splits=3, shuffle = True)

scores = cross_val_score(
    model,
    X_train_transformed,
    y_train,
    cv=KFold(5,shuffle=True),
    scoring='r2') # StratifiedKfold used for classification problems

In [55]:
#is there a score method used in the lesson I am missing? ok so this is accuracy but uh... need to confirm why
scores = cross_val_score(general_pipeline, X_train, y_train, cv=5)

In [56]:
print(f'Validation accuracy scores: {np.round(scores, 5)}')
print(f'\nmean: {scores.mean():.2}')
print(f'\nstd: {scores.std():.2}')

Validation accuracy scores: [0.76866 0.80597 0.75373 0.84211 0.80451]

mean: 0.79

std: 0.031


In [57]:
#gonna see if I can pull these out per the bonus challenge in 2.5, ROC *might* follow, this is actually complicated because I cross validated the log reg
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

In [59]:
from sklearn.ensemble import RandomForestClassifier

general_pipeline_t = Pipeline([
    ('column_transformer', column_transformer),
    ("model", RandomForestClassifier())
])

In [60]:
general_pipeline_t.fit(X_train, y_train)


Pipeline(steps=[('column_transformer',
                 ColumnTransformer(transformers=[('one_encode', OneHotEncoder(),
                                                  ['Pclass', 'Sex']),
                                                 ('pipeline',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('bins',
                                                                   KBinsDiscretizer())]),
                                                  ['Age']),
                                                 ('pass', 'passthrough',
                                                  ['SibSp', 'Parch'])])),
                ('model', RandomForestClassifier())])

In [61]:
general_pipeline_t.score(X_test, y_test)

0.8295964125560538